In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import pylab
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats import norm
import statsmodels.api as sm

Dowload hourly price of a stock and daily SNB rates

In [ ]:
SNB_data = pd.read_excel('current_interest_rates.xlsx',index_col=0)
SNB_data=SNB_data.sort_index().SNBLZ
SNB_data.index=SNB_data.index.tz_localize(None)

data = yf.download(tickers='UBSG.SW',start=SNB_data.index[0],end=SNB_data.index[-1], interval='1h')
close = data.Close
close.plot()

[*********************100%***********************]  1 of 1 completed


<AxesSubplot:xlabel='Datetime'>

Find the time series of daily returns and concat with SNB_data to match the days

In [ ]:
R = np.log(np.exp(1+close.pct_change().dropna()).resample('d').mean())-1
R.index=R.index.tz_localize(None)

full_data = pd.concat([R, SNB_data], axis=1).dropna()

find OLS estimator

In [ ]:
Y = full_data.Close.values[1:]
X = np.ones((len(Y),3))
X[:,1] = full_data.Close.values[:-1]
X[:,2] = full_data.SNBLZ.values[:-1]

X = np.matrix(X)
Y = np.matrix(Y).transpose()

np.linalg.inv(X.transpose()*X)*X.transpose()*Y

In [ ]:
Y = full_data.Close.values[1:]
X = np.ones((len(Y),3))
X[:,1] = full_data.Close.values[:-1]
X[:,2] = full_data.SNBLZ.values[:-1]

model = sm.OLS(Y, X).fit()

#view model summary
print(model.summary())